In [3]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import yfinance as yf



In [4]:

# Define tickers
tickers = ['msft', 'aapl', 'goog', 'nvda', 'meta', 'tsm','orcl', 'adbe', 'crm', 'csco', 'tmus', 'intc', 'intu', 'ibm', 'tsla', 't', 'now', 'adi', 'uber', 'shop', 'abnb', 'pypl', 'infy', 'snow', 'wday', 'aph']

# Create an empty list to store the dataframes
df_list = []

# Loop over each ticker in the list and download the data
for ticker in tickers:
    ticker_object = yf.Ticker(ticker)

    # Download the data
    ticker_df = ticker_object.history(period="5y")
    
    # Add a column to identify the ticker
    ticker_df['Ticker'] = ticker 
        
    # Append the DataFrame to the list
    df_list.append(ticker_df)

# Concatenate all the dataframes in the list into one DataFrame
stock_info = pd.concat(df_list)

# Display the first 5 rows
stock_info.tail()


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
Date,,,,,,,,
2023-07-11 00:00:00-04:00,84.930000,85.040001,84.000000,84.849998,1893800,0.0,0.0,aph
2023-07-12 00:00:00-04:00,85.500000,85.809998,84.910004,85.559998,2147700,0.0,0.0,aph
2023-07-13 00:00:00-04:00,85.730003,85.889999,85.309998,85.720001,2091900,0.0,0.0,aph
2023-07-14 00:00:00-04:00,85.559998,85.669998,84.160004,84.610001,1811100,0.0,0.0,aph
2023-07-17 00:00:00-04:00,84.570000,85.330002,84.320000,85.180000,2067800,0.0,0.0,aph


In [5]:
# Create a new list of column names with 'Ticker' at the start
cols = ['Ticker'] + [col for col in stock_info if col != 'Ticker']

# Reindex the DataFrame to rearrange the columns
stock_info = stock_info[cols]

# Sort the DataFrame by the 'Ticker' column
stock_info = stock_info.sort_values(['Date', 'Ticker'])

# Display the first 50 rows

stock_info.head(100)




,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2018-07-18 00:00:00-04:00,aapl,45.819860,45.824639,45.377858,45.490150,65573600,0.0,0.0
2018-07-18 00:00:00-04:00,adbe,257.859985,260.940002,256.540009,259.779999,2637000,0.0,0.0
2018-07-18 00:00:00-04:00,adi,89.540495,90.331122,89.485971,90.049400,1635200,0.0,0.0
2018-07-18 00:00:00-04:00,aph,42.327865,42.536986,42.199542,42.361134,1908200,0.0,0.0
2018-07-18 00:00:00-04:00,crm,147.020004,148.190002,145.759995,147.839996,2908100,0.0,0.0
...,...,...,...,...,...,...,...,...
2018-07-24 00:00:00-04:00,aph,42.033188,42.456181,41.990411,42.242306,3216200,0.0,0.0
2018-07-24 00:00:00-04:00,crm,148.550003,148.850006,145.179993,146.000000,4052300,0.0,0.0
2018-07-24 00:00:00-04:00,csco,36.295807,36.863198,36.166854,36.441952,17493900,0.0,0.0


In [6]:
# Reset the index
stock_info = stock_info.reset_index()

# Ensure the 'Date' column is a datetime object
stock_info['Date'] = pd.to_datetime(stock_info['Date'])

# Set 'Date' as the index again (this is necessary for resampling)
stock_info.set_index('Date', inplace=True)

# Create a grouper that groups by 'Ticker' and by month
grouped = stock_info.groupby('Ticker').resample('M')

# Compute the mean of the 'Close' price for each group
monthly_stock_info = grouped.mean()

# Reset the index again so that 'Date' and 'Ticker' are regular columns
monthly_stock_info.reset_index(inplace=True)

# Display the first rows of the DataFrame
monthly_stock_info.head(50)


,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,aapl,2018-07-31 00:00:00-04:00,45.902041,46.176798,45.536734,45.837533,8.490244e+07,0.000000,0.000000
1,aapl,2018-08-31 00:00:00-04:00,50.892733,51.389504,50.609906,51.098368,1.217946e+08,0.007935,0.000000
2,aapl,2018-09-30 00:00:00-04:00,53.305370,53.832708,52.765160,53.243664,1.429415e+08,0.000000,0.000000
3,aapl,2018-10-31 00:00:00-04:00,53.034190,53.691332,52.236842,52.949233,1.373476e+08,0.000000,0.000000
4,aapl,2018-11-30 00:00:00-05:00,46.109031,46.620922,45.359189,45.968402,1.831098e+08,0.008690,0.000000
5,aapl,2018-12-31 00:00:00-05:00,39.756396,40.256071,38.949903,39.521375,1.892468e+08,0.000000,0.000000
6,aapl,2019-01-31 00:00:00-05:00,36.957192,37.419590,36.557578,37.091351,1.577309e+08,0.000000,0.000000
7,aapl,2019-02-28 00:00:00-05:00,41.366249,41.725548,41.158623,41.447228,9.948223e+07,0.009605,0.000000
8,aapl,2019-03-31 00:00:00-04:00,44.271483,44.684202,43.833451,44.288397,1.239965e+08,0.000000,0.000000
9,aapl,2019-04-30 00:00:00-04:00,48.292815,48.706110,47.994236,48.449757,9.640337e+07,0.000000,0.000000


In [7]:
# Round the 'Volume' column to the nearest million
monthly_stock_info['Volume'] = (monthly_stock_info['Volume'] / 1_000_000).round().astype(int)

# Drop the specified columns
monthly_stock_info = monthly_stock_info.drop(['Dividends', 'Stock Splits'], axis=1)

# Remove all null values from the DataFrame
monthly_stock_info = monthly_stock_info.dropna()

# Format all decimal values to two decimal places
monthly_stock_info = monthly_stock_info.round(decimals=2)

# Display the first rows of the DataFrame
monthly_stock_info.head()


,Ticker,Date,Open,High,Low,Close,Volume
0,aapl,2018-07-31 00:00:00-04:00,45.90,46.18,45.54,45.84,85
1,aapl,2018-08-31 00:00:00-04:00,50.89,51.39,50.61,51.10,122
2,aapl,2018-09-30 00:00:00-04:00,53.31,53.83,52.77,53.24,143
3,aapl,2018-10-31 00:00:00-04:00,53.03,53.69,52.24,52.95,137
4,aapl,2018-11-30 00:00:00-05:00,46.11,46.62,45.36,45.97,183


In [8]:
# Export Data to csv

# Select columns to export
OpenClose_df = monthly_stock_info[['Ticker', 'Date', 'Open', 'Close']]

path = "OpenClose.csv"

# Export the DataFrame to a csv file
OpenClose_df.to_csv(path, index=False)


PermissionError: [Errno 13] Permission denied: 'OpenClose.csv'

In [9]:
# Select columns to export
Volume_df= monthly_stock_info[['Ticker', 'Date', 'Volume']]

path = "Volume.csv"

# Export the DataFrame to a csv file
Volume_df.to_csv(path, index=False)


PermissionError: [Errno 13] Permission denied: 'Volume.csv'

In [17]:
HighLow_df = monthly_stock_info[['Ticker', 'Date', 'High', 'Low']]

path = "HighLow.csv"

# Export the DataFrame to a csv file
HighLow_df.to_csv(path, index=False)


In [11]:
# Export Data to tradinginfo.csv

# Select columns to export
TradingInfo_df = monthly_stock_info[['Ticker', 'Date', 'Low', 'Open', 'Close', 'High']]

path = "TradingInfo.csv"

# Export the DataFrame to a csv file
TradingInfo_df.to_csv(path, index=False)

In [9]:
# Define tickers
tickers = ['msft', 'aapl', 'goog', 'nvda', 'meta', 'tsm', 'avgo', 'orcl', 'adbe', 'crm', 'csco', 'tmus', 'intc', 'intu', 'ibm', 'tsla', 't', 'now', 'adi', 'uber', 'shop', 'abnb', 'pypl', 'infy', 'snow', 'wday', 'aph']

# Create an empty list to store the information DataFrames
df_list = []

# Loop over each ticker in the list
for ticker in tickers:
    ticker_object = yf.Ticker(ticker)
  
    # Get the company info
    info = ticker_object.info

    # Convert the info into a DataFrame
    info_df = pd.DataFrame.from_dict(info, orient='index').T

    # Append the DataFrame to the list
    df_list.append(info_df)

# Concatenate all the dataframes in the list into one DataFrame
company_info_df = pd.concat(df_list, ignore_index=True)

# Display the first rows of the DataFrame
company_info_df.head(50)



,address1,city,state,zip,country,phone,fax,website,industry,industryDisp,...,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio,address2,industrySymbol
0,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,425 706 7329,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,...,83441000448,0.104,0.071,0.68522,0.4821,0.41415,USD,2.4389,NaN,NaN
1,One Apple Park Way,Cupertino,CA,95014,United States,408 996 1010,NaN,https://www.apple.com,Consumer Electronics,Consumer Electronics,...,109583998976,0.0,-0.025,0.43181,0.32145,0.29163,USD,2.5214,NaN,NaN
2,1600 Amphitheatre Parkway,Mountain View,CA,94043,United States,650 253 0000,NaN,https://www.abc.xyz,Internet Content & Information,Internet Content & Information,...,89898000384,-0.047,0.026,0.55302,0.30742,0.25355,USD,1.5415,NaN,NaN
3,2788 San Tomas Expressway,Santa Clara,CA,95051,United States,408 486 2000,NaN,https://www.nvidia.com,Semiconductors,Semiconductors,...,6821000192,0.281,-0.132,0.56314,0.23534,0.17374,USD,2.2442,NaN,NaN
4,1601 Willow Road,Menlo Park,CA,94025,United States,650 543 4800,NaN,https://investor.fb.com,Internet Content & Information,Internet Content & Information,...,50396999680,-0.191,0.026,0.79576,0.3618,0.28465,USD,0.9394,NaN,NaN
5,Hsinchu Science Park,Hsinchu City,NaN,300096,Taiwan,886 3 563 6688,886 3 563 7000,https://www.tsmc.com,Semiconductors,Semiconductors,...,1623674191872,0.021,0.036,0.59684,0.68252,0.49474,TWD,4.3744,"No. 8, Li-Hsin Road 6",NaN
6,1320 Ridder Park Drive,San Jose,CA,95131-2313,United States,408 433 8000,NaN,https://www.broadcom.com,Semiconductors,Semiconductors,...,NaN,NaN,NaN,0.0,0.0,0.0,USD,1.4913,NaN,NaN
7,2300 Oracle Way,Austin,TX,78741,United States,737 867 1000,NaN,https://www.oracle.com,Software—Infrastructure,Software—Infrastructure,...,15502999552,-0.19,0.179,0.74496,0.38792,0.29157,USD,2.2782,NaN,NaN
8,345 Park Avenue,San Jose,CA,95110-2704,United States,408 536 6000,408 537 6000,https://www.adobe.com,Software—Infrastructure,Software—Infrastructure,...,7761999872,0.019,0.092,0.8766,0.37341,0.33913,USD,2.0519,NaN,NaN
9,Salesforce Tower,San Francisco,CA,94105,United States,415 901 7000,415 901 7040,https://www.salesforce.com,Software—Application,Software—Application,...,7926000128,6.079,0.113,0.73779,0.21396,0.09199,USD,1.438,3rd Floor 415 Mission Street,h885


In [10]:
df_list = []

# Loop over each ticker in the list
for ticker in tickers:
    ticker_object = yf.Ticker(ticker)
  
    # Get the company info
    info = ticker_object.info

    # Add the ticker to the info
    info['Ticker'] = ticker

    # Convert the info into a DataFrame
    info_df = pd.DataFrame.from_dict(info, orient='index').T

    # Append the DataFrame to the list
    df_list.append(info_df)

# Concatenate all the dataframes in the list into one DataFrame
company_info_df = pd.concat(df_list, ignore_index=True)

# Specify the columns you want to keep, now including 'Ticker'
columns_to_keep = ['Ticker', 'longName', 'city', 'country', 'website', 'industry', 'revenueGrowth', 'ebitdaMargins']

# Filter the DataFrame
filtered_df = company_info_df[columns_to_keep]
filtered_df = filtered_df.rename(columns={'longName': 'CompanyName'})

# Drop rows with null values
filtered_df = filtered_df.dropna()

# Sort by Ticker
filtered_df = filtered_df.sort_values(['Ticker'])

# Display the first rows of the DataFrame
filtered_df.head(50)



,Ticker,CompanyName,city,country,website,industry,revenueGrowth,ebitdaMargins
1,aapl,Apple Inc.,Cupertino,United States,https://www.apple.com,Consumer Electronics,-0.025,0.32145
21,abnb,"Airbnb, Inc.",San Francisco,United States,https://www.airbnb.com,Travel Services,0.205,0.22151
8,adbe,Adobe Inc.,San Jose,United States,https://www.adobe.com,Software—Infrastructure,0.092,0.37341
18,adi,"Analog Devices, Inc.",Wilmington,United States,https://www.analog.com,Semiconductors,0.098,0.5255
26,aph,Amphenol Corporation,Wallingford,United States,https://www.amphenol.com,Electronic Components,0.007,0.23812
9,crm,"Salesforce, Inc.",San Francisco,United States,https://www.salesforce.com,Software—Application,0.113,0.21396
10,csco,"Cisco Systems, Inc.",San Jose,United States,https://www.cisco.com,Communication Equipment,0.135,0.29517
2,goog,Alphabet Inc.,Mountain View,United States,https://www.abc.xyz,Internet Content & Information,0.026,0.30742
14,ibm,International Business Machines Corporation,Armonk,United States,https://www.ibm.com,Information Technology Services,0.004,0.2087
23,infy,Infosys Limited,Bengaluru,India,https://www.infosys.com,Information Technology Services,0.064,0.23078


In [11]:
import os

directory = "../Database"
if not os.path.exists(directory):
    os.makedirs(directory)

path = "stock_info.csv"

# Export the DataFrame to a csv file
filtered_df.to_csv(path, index=False)
